# GMM over z

In this notebook, we **fit a generative model over latent variables “z”** provided by the pre-trained encoder network in Task2, in order to **define a probability distribution over “z”, and then to retrieve a complete generative mechanism and sample new sounds**.

We use the package [sklearn.mixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) to help us build a GMM generative model

<p><b> GMM methods </b></p>

<img src="pictures/GMM_methods.jpg" alt="GMM methods in sklearn" width="800">

In [1]:
import numpy as np
import sklearn
from sklearn.mixture import GaussianMixture

print(sklearn.__version__)

0.24.1


## Import and reshape the z dataset

[encoder.py](https://github.com/magenta/ddsp/blob/master/ddsp/training/encoders.py)

[decoder.py](https://github.com/magenta/ddsp/blob/master/ddsp/training/decoders.py)

In [2]:
load_path = 'z_datasets/z_piano_ae.npy'
X = np.load(load_path)
print(np.shape(X))

(15, 1, 15000, 16)


#### Reshape the dataset as two-dimensional

The original dataset is three-dimensional. We reduce its dimensionality to 2 in order to fit the GMM.

In [3]:
# reduce dimensionality 
N, T_step, Z_dim = np.shape(X)[0], np.shape(X)[2], np.shape(X)[3]
X_new = np.zeros(shape=(N,T_step, Z_dim))

for i in np.arange(N):
    X_new[i] = X[i,0]
    
print(np.shape(X_new))
#print(X_new)

X_re = np.reshape(X_new,(N*T_step,16))
print('\n',np.shape(X_re),'\n',X_re)

(15, 15000, 16)

 (225000, 16) 
 [[-0.03763556 -0.207986   -0.04950411 ...  0.08215524 -1.41192687
   0.34535056]
 [-0.00612059 -0.23088677 -0.04997538 ...  0.11514032 -1.46286297
   0.34728429]
 [ 0.02539439 -0.25378755 -0.05044664 ...  0.1481254  -1.51379919
   0.34921801]
 ...
 [ 0.29531065  1.9499706  -1.02815902 ... -0.57737291  0.40595052
   0.21172309]
 [ 0.29531065  1.9499706  -1.02815902 ... -0.57737291  0.40595052
   0.21172309]
 [ 0.29531065  1.9499706  -1.02815902 ... -0.57737291  0.40595052
   0.21172309]]


In [4]:
# D = np.reshape(X_re,(N,T_step, Z_dim))
# print(X == D)

## Fit the dataset z

In [5]:
# fit Z and build a GMM model
gm = GaussianMixture(n_components=10).fit(X_re)
# attributes of the model
"""
weights_: array-like of shape (n_components,)
The weights of each mixture components.

means_: array-like of shape (n_components, n_features)
The mean of each mixture component.
"""

means = gm.means_
weights = gm.weights_

print('means:',means,'\n','weights of each component:',weights)

means: [[-1.22767301e+00 -1.28812173e-01  1.77424480e-01 -9.25735158e-01
   6.77147080e-01 -3.31983194e-01  1.40209803e+00 -6.74201057e-01
   5.39708219e-01  5.50122747e-01  8.12741567e-02  5.44950695e-01
  -1.13531327e+00  2.45022098e-01 -3.01123957e-01 -9.93759289e-01]
 [ 7.96448686e-02 -5.64504324e-02  1.56221231e-01 -4.73284887e-02
   1.11633514e+00 -3.72630153e-01 -8.27748364e-01  2.31575139e-01
   4.74906121e-01  9.19264573e-01 -1.49978262e-01 -2.82553749e-01
  -5.80626206e-02 -2.68634055e-01  8.81077120e-01  2.26825757e-02]
 [ 8.78919302e-01 -5.53427981e-01  2.74115164e-02  2.30283069e-01
  -8.30643824e-01  2.82540548e-01  4.27097538e-01  1.74838354e-01
   9.35449114e-01  4.31954232e-01 -8.36768799e-01  5.11287448e-01
  -5.49325498e-01  9.11457546e-01 -1.13178474e+00 -7.52935361e-01]
 [ 1.20971817e+00 -8.94982682e-01 -1.03763963e+00  5.24424764e-01
  -1.96524315e-01 -1.13555423e-01  3.81952781e-02 -2.14981808e-01
  -2.63100764e-01 -6.33543747e-01  2.46291777e-01 -7.55363364e-01


In [6]:
predictions = gm.sample(1000)

print('\n', np.shape(predictions[0]))
print('\n', predictions[0])


 (1000, 16)

 [[-1.53039119  0.73450421  0.08220713 ... -0.02114236 -0.39266324
  -1.05753607]
 [-1.90418642 -1.20635352 -0.09904436 ...  0.94369902  0.1354099
   0.26751625]
 [-0.62928271 -1.14141235  0.21972873 ...  1.64428289  0.16504005
   0.23592995]
 ...
 [-0.38301217 -0.72798212  1.40531473 ...  1.48213132  0.18790059
  -2.11798967]
 [-0.098572   -1.4043831   0.77956941 ...  1.59625837  0.48671803
   0.2077207 ]
 [ 0.72064548 -2.53069686  0.37502832 ...  0.16908954 -1.37669371
  -0.55005173]]
